In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.spatial import distance_matrix

np.random.seed(123)

# Load Data

In [ ]:
data = pd.read_csv(r'./maps/FVG/FVG_companies.csv', dtype='str')
data.columns = ['ind', 'cf', 'company','unit', 'lat', 'lon']
data['lat'] = data['lat'].astype(float)
data['lon'] = data['lon'].astype(float)
data.shape

# Data Exploration

In [ ]:
def plot_clusters(X, cluster_labels, title=""):
    for i in np.unique(cluster_labels):
        cluster_points = X[cluster_labels == i , :]       
        if i == 0: 
            ax.scatter(cluster_points[:,0] , cluster_points[:,1] , color="yellow", marker='.', s=2)
        else:
            ax.scatter(cluster_points[:,0] , cluster_points[:,1] , cmap = i, marker='o', s=2)
    ax.set_title(title, fontweight='bold')
    return

In [ ]:
X = np.array(data[ ['lon','lat'  ] ]) 
n, p  = X.shape
_, ax = plt.subplots(figsize=(8,8))
plot_clusters(X, np.ones(n), title = "dataset")
plt.show()


## Scaling factor

In [ ]:
# approximated, valid only at short distances but good enough to give a rougn idea of epsilon
R = 6371 #earth radius
lon1, lat1,  =  12.6, 46.0
lon2, lat2   =  13.8, 46.0
# longitude x
dx = R*(lon2-lon1)*np.cos(lat1)*np.pi/180
# latitude y
dy = R* (lat2-lat1)*np.pi/180
d =  np.sqrt(dx**2 +  dy**2)
dx, dy

In [ ]:
lon1, lat1,  =  13.2, 45.6
lon2, lat2   =  13.2, 46.6
# longitude x
dx = R*(lon2-lon1)*np.cos(lat1)*np.pi/180
# latitude y
dy = R* (lat2-lat1)*np.pi/180
d =  np.sqrt(dx**2 +  dy**2)
dx, dy

In [ ]:
# TODO add scaling factor 57,111 to have more "readable" values of epsilon in km

## calculate density for each point
X = data points = company position
G = grid points equally spaced

In [ ]:
grid = pd.read_csv(r'./maps/FVG/FVG_grid.csv')
nbins=np.sqrt(grid.shape[0]).astype(int)
xi = np.array(grid.x).reshape(nbins,nbins)
yi = np.array(grid.y).reshape(nbins,nbins)
G =  np.array(grid)

In [ ]:
n_reshape = np.sqrt(n_grid_points).astype(int)

# calculate distance matrix

In [ ]:
dm = distance_matrix(G,X)
n_data_points= X.shape[0]
n_grid_points=G.shape[0]

In [ ]:
    # n_grid_points = G.shape[0]
    # dm = distance_matrix(G,X)
    # eps_neighb=np.empty(n_grid_points)
    # epsilon=.7
    # for g in range(n_grid_points):
    #     print(g)
    #     distances = dm[g]
    #     eps_neighb[g] = np.where(X[ (distances < epsilon) ])[0]

    # return eps_neighb


In [ ]:
def epsilon_neighborhood(dm, X, g, epsilon):
    """
    dm: distance_matrix(G,X)
    g: point of the grid G
    returns eps_neighb a list of indices of X that are within a distance epsilon of point G
    """
    distances = dm[g]
    eps_neighb = np.where(X[ (distances < epsilon) ])[0]
    number_of_neighb = len(eps_neighb)
    
    return number_of_neighb, eps_neighb


In [ ]:
minPoints = 20
fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize = (15,5))
nplot=0
for epsilon in [1e-3, 1e-2, 5e-2, 1e-1, 2e-1]:
    nn_epsilon = []
    is_core = []
    for g in range(n_grid_points):
        number_of_neighb, eps_neighb = epsilon_neighborhood(dm, X, g, epsilon)
        nn_epsilon.append(number_of_neighb)
        is_core.append( number_of_neighb > minPoints  )
    
    ax[nplot].hist(nn_epsilon, bins = 100)
    nplot+=1
    fraction_core_points = np.array(is_core).mean()
    print(f'Given Epsilon = {epsilon} and minPoints = {minPoints}, the fraction of core points is {fraction_core_points}')

select epsilon = .05, minPoints=20

# compute density

In [ ]:
def compute_density(dm, X, G, epsilon):
    n_grid_points=  G.shape[0]
    n_data_points = X.shape[0]
    density = np.zeros(n_grid_points).astype(int)
    number_of_neighb = np.zeros(n_grid_points).astype(int)

    for g in range(n_grid_points):
        nnb, eps_neighb = epsilon_neighborhood(dm, X, g, epsilon)
        number_of_neighb[g] = nnb
    
    density = number_of_neighb / n_data_points
        
    return np.array(number_of_neighb), np.array(density)

In [ ]:

nn, dG1 = compute_density(dm, X, G,  epsilon=2e-2)


In [ ]:
cmap = "Reds"

In [ ]:
dG1r = dG1.reshape((n_reshape,n_reshape))
plt.pcolormesh(xi, yi, dG1r,shading='auto', cmap=cmap)


In [ ]:

fig, (ax1,ax2, ax3) = plt.subplots(figsize=(16,5), nrows=1, ncols=3)
nn, dG1 = compute_density(dm, X, G, epsilon=0.06)

cmap="Reds"
n_reshape = 100
dG1r = dG1.reshape((n_reshape,n_reshape))

ax1.pcolormesh(xi, yi, dG1r,shading='nearest', cmap=cmap)

threshold=epsilon/50
dG1rM = np.ma.masked_array(dG1r, dG1r <threshold)
ax2.pcolormesh(xi, yi, dG1rM, shading='auto', cmap=cmap, vmin=0)

ax3.pcolormesh(xi, yi, dG1rM, shading='auto', cmap=cmap, alpha = 1, zorder = 1)
ax3.contour(xi, yi, dG1rM, cmap="magma", vmin=threshold, zorder = 3)
plt.show()


In [ ]:
def plot_map(df, color_fill = "green", alpha = .2, color_border= "gray" , linewidth = .8):
    for _, row in df.iterrows():
        x, y = list(zip(*row.poly)) 
        plt.fill(x,y, color = color_fill, alpha = alpha, zorder=1)
        plt.plot(x,y, color = color_border, linewidth=linewidth, zorder=2)

In [ ]:
#load data from file
ita = pd.read_json(r'./maps/FVG/FVG_mappa.json', orient='records')
# filter a single region
fvg=ita[ ita.istat_r==6]
# plot the map


In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
plot_map(fvg, color_fill = "lightgray", alpha = 1, color_border= "darkgray" , linewidth = 2)
plt.pcolormesh(xi, yi, dG1rM, shading='auto', cmap=cmap, alpha = .5, zorder = 3)
plt.contour(xi, yi, dG1rM, cmap="magma", vmin=threshold, zorder = 4)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plot_map(fvg, color_fill = "lightgray", alpha = .6, color_border= "white" , linewidth = .8)
plt.contour(xi, yi, dG1rM, cmap="magma", vmin=threshold, zorder = 4)

plt.show()

## other data points

In [ ]:
dd = pd.read_csv(r'./data/summary.csv')
dd = dd[ dd.x > 12.3]
dd = dd[ dd.x < 14.0]
dd = dd[ dd.y > 45.6]
dd = dd[ dd.y < 46.7]



In [ ]:
D = np.array(dd[ ['x','y' ] ]) 

In [ ]:
dm = distance_matrix(G,D)
n_data_points= D.shape[0]
n_grid_points= G.shape[0]
n_reshape = np.sqrt(n_grid_points).astype(int)

In [ ]:
D.shape

In [ ]:
G.shape

In [ ]:
dm.shape

In [ ]:
nn, dens = compute_density(dm, D, G,  epsilon=5e-2)



In [ ]:
dens_r = dens.reshape((n_reshape,n_reshape))
plt.pcolormesh(xi, yi, dens_r, shading='auto', cmap="magma")

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
cmap = "magma"
plot_map(fvg, color_fill = "lightgray", alpha = 1, color_border= "darkgray" , linewidth = 2)

threshold=epsilon/100
dens_r_M = np.ma.masked_array(dens_r, dG1r <threshold)
plt.pcolormesh(xi, yi, dens_r_M, shading='auto', cmap=cmap, alpha = .5, zorder = 4)

plt.contour(xi, yi, dens_r_M, cmap=cmap,  zorder = 5)


plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))

plot_map(fvg, color_fill = "lightgray", alpha = .2, color_border= "darkgray" , linewidth = 2)

plt.scatter(dd.x,dd.y, s=dd.z*10, c = dd.z, alpha = .8, cmap = "magma", zorder = 5)


# CLUSTERING

In [ ]:
epsilon = 1
minPoints = 10
dm = distance_matrix(X,X)
is_core = np.zeros(n).astype(int)
for j in range(n):
    n_neibs = np.unique(np.where( dm[j] < epsilon )).flatten()
    if len(n_neibs) >= minPoints: is_core[j] = 1


In [ ]:
#compute the minimum distances of the k-th nearest neighbour
from scipy.spatial import distance_matrix
dm = distance_matrix(X,X)
values = np.reshape(dm,-1)
plt.hist(values, bins = 100)
plt.show()


In [ ]:
knn = 5
min_distances= np.partition(dm, knn, axis=0)[knn] 
plt.hist(min_distances, bins = 100)
plt.show

In [ ]:
for p in range(10,100,10):
    print(f"il {p}° percentile ==> distanza {np.percentile(min_distances, p)}")

In [ ]:
# dm = distance_matrix(X,X)
# index = 2
# distances = dm[index]

# epsilon = .02

# distances_epsilon = distances[ distances <= epsilon]



In [ ]:
# def epsilon_neighborhood( index, epsilon):
#     distances = dm[index]
#     neighbours_indices = np.where( distances <= epsilon )
#     return np.array(neighbours_indices)[0]

# def is_core(p, epsilon, minPoints):
#     nbhood =  epsilon_neighborhood(p, epsilon)
#     return (len(nbhood) >= minPoints)

# def is_directly_density_reachable(p,q, epsilon, minPoints):
#     '''An object q is directly density-reachable from object p 
#     if q is within the ε-Neighborhood of p 
#     and p is a core object'''
#     is_ddr = False
#     neib=np.array([])
#     if is_core(p, epsilon, minPoints):
#         neib = epsilon_neighborhood(p, epsilon)
#         if (q in neib):
#             is_ddr = True
#     return is_ddr, neib




In [ ]:
# for eps in [.11, .002]:
#     nbindexes = epsilon_neighborhood(12, eps)
#     print(nbindexes)
    

In [ ]:
# for i in range(70):
#     nbindexes =  epsilon_neighborhood(i, epsilon=.0011)
#     print( len(nbindexes))
    

In [ ]:
# epsilon = .0011
# minPoints = 26
# t,f= 0,0
# for p in range(100):
#     for q in range(100):
#         is_ddr, neib = is_directly_density_reachable(p,q, epsilon, minPoints)
#         if is_ddr: 
#             t+=1 
#         else: 
#             f+=1

# print(t,f)

In [ ]:
#retrieva all points density reachable from P

# def density_reachable( p, epsilon, minPoints):
#     density_reachable = []
#     for q in list(points):
#         is_ddr, neib = is_directly_density_reachable(p,q, epsilon, minPoints)
#         if is_ddr: 
#             density_reachable.append(neib)
#     return density_reachable



In [ ]:

def clustering_DBSCAN_noise(X, epsilon, minPoints):
    n_points = len(X)
    dm = distance_matrix(X,X)
    # identify core points 
    is_core = np.zeros(n_points).astype(int)
    for j in range(n_points):
        n_neibs = np.unique(np.where( dm[j] < epsilon )).flatten()
        if len(n_neibs) >= minPoints: is_core[j] = 1
 
    
    points = np.arange(n_points)
    clusters = np.zeros(n_points)

    cluster_id = 0
    tovisit = np.sum(clusters == 0)
    while(tovisit > 0):
        is_core_unvisited = ((clusters == 0) & (is_core == 1))
        p = np.random.choice(points[ is_core_unvisited ])
        # directly density reachable!
        nbhood = np.unique(np.where( dm[p] <= epsilon )).flatten()
        #check indirect reachability  
        
        old_nb_len = 0
        nb_len = len(nbhood)
        while (nb_len>old_nb_len): 
            for q in nbhood: 

                indirect = np.unique(np.where( dm[q] <= epsilon )).flatten()
                nbhood = np.unique(np.append(nbhood, indirect).flatten())
            # add here a condition to keep searching further levels  
            old_nb_len = nb_len
            nb_len = len(nbhood)
  
             
                
        if (len(nbhood) >= minPoints):# is_core:
            cluster_id += 1
            clusters[p] = int(cluster_id)
            for q in nbhood:
                    clusters[q] = int(cluster_id)
        else: #is halo
            clusters[p] = int(-1)

        is_core_unvisited = ((clusters == 0) & (is_core == 1))
        tovisit = len(points[is_core_unvisited])
    
    #encode "halo" as 0    
    clusters[ clusters == -1] = 0
    return clusters
    

In [ ]:
epsilon = .02
minPoints= 20
np.random.seed(123)
clusters = clustering_DBSCAN_noise(X, epsilon=epsilon, minPoints=minPoints)
print((set(clusters)))

data['cluster']=clusters.astype(int)
data = data[ [ 'cf', 'company', 'lat', 'lon', 'cluster'] ]
data.to_csv('./data/FVG_comp_clusters.csv', index=False)

plt.figure(figsize=(10,10))

for clust in list(set(data.cluster)):
    x = np.array(data.lon[ data.cluster == clust ])
    y = np.array(data.lat[ data.cluster == clust ])
    if clust == 0: 
        plt.scatter(x, y, s=10, marker = ".", color = "yellow", label="halo")
    else:
        plt.scatter(x, y, s=20, marker = "X", label=f"Cluster{clust}")

plt.title('DBSCAN - my implementation')
plt.xlabel('X')
plt.ylabel('Y')

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plot_map(fvg, color_fill = "lightgray", alpha = .6, color_border= "white" , linewidth = .8)
for clust in list(set(data.cluster)):
    x = np.array(data.lon[ data.cluster == clust ])
    y = np.array(data.lat[ data.cluster == clust ])
    if clust == 0: 
        plt.scatter(x, y, s=10, marker = ".", color = "gray",label="halo", zorder=3)
    else:
        plt.scatter(x, y, s=20, marker = "X",  label=f"Cluster{clust}", zorder=4)


In [ ]:

# np.random.seed(999)
# clusters = clustering_DBSCAN_noise(X, epsilon=epsilon, minPoints=minPoints)
# print((set(clusters)))

# data['cluster']=clusters.astype(int)
# data = data[ [ 'cf', 'company', 'lat', 'lon', 'cluster'] ]
# data.to_csv('./data/FVG_comp_clusters.csv', index=False)

# plt.figure(figsize=(10,10))

# for clust in list(set(data.cluster)):
#     x = np.array(data.lon[ data.cluster == clust ])
#     y = np.array(data.lat[ data.cluster == clust ])
#     if clust == 0: 
#         plt.scatter(x, y, s=10, marker = ".", color = "yellow", label="halo")
#     else:
#         plt.scatter(x, y, s=20, marker = "X", label=f"Cluster{clust}")


# #plt.legend([f"Cluster {clust}" for clust in np.unique(cluster)], loc ="lower right")
# #plt.legend()
# plt.title('DBSCAN - my implementation')
# plt.xlabel('X')
# plt.ylabel('Y')

In [ ]:
from sklearn.cluster import DBSCAN
epsilon = 2.2
minPoints= 20

clustering = DBSCAN(eps=epsilon, min_samples=minPoints).fit(X)
cc = clustering.labels_
plt.figure(figsize=(10,7))
for clust in list(set(cc)):
    x = np.array(data.lon[ clust == cc ])
    y = np.array(data.lat[ clust == cc ])
    if clust == -1: 
        plt.scatter(x, y, s=10, marker = ".", color = "yellow", label="halo")
    else:
        plt.scatter(x, y, s=20, marker = "X", label=f"Cluster{clust}")


#plt.legend([f"Cluster {clust}" for clust in np.unique(cluster)], loc ="lower right")
#plt.legend()
plt.title('DBSCAN - my CLASS implementation')
plt.xlabel('X')
plt.ylabel('Y')

print(len(set(cc)))

In [ ]:
print(":-D")

Silhouette score
eg from here
https://gist.github.com/AlexandreAbraham/5544803?msclkid=57d3c2cfb28b11ec853386a9714df71b


calculate comparison (precision, recall) as in this
https://towardsdatascience.com/evaluating-clustering-results-f13552ee7603


In [ ]:
from sklearn.metrics.cluster import fowlkes_mallows_score

np.random.seed(999)
clusters1 = clustering_DBSCAN_noise(X, epsilon=.0015, minPoints=20)
plot_clusters(X, clusters1)

clusters2 = clustering_DBSCAN_noise(X, epsilon=.0010, minPoints=20)
plot_clusters(X, clusters2)

clusters3 = clustering_DBSCAN_noise(X, epsilon=.0005, minPoints=20)
plot_clusters(X, clusters3)

plt.show()
fowlkes_mallows_score(clusters2, clusters1)

https://geodacenter.github.io/workbook/99_density/lab9b.html?msclkid=d7e2bc81b28d11ec92fcd58b49d1b2ba

Laio / Peak Halo
https://people.sissa.it/~laio/Research/Res_clustering.php
file:///C:/Users/morea/Downloads/Clustering_by_fast_search_and_find_of_density_peaks.pdf


https://github.com/markovmodel/PyEMMA/issues/998?msclkid=96f31b4db28d11ec985ce9b15c28caff

Rodriguez, A., & Laio, A. (2014). Clustering by fast search and find of density peaks. Science, 344(6191), 1492-1496. doi:10.1126/science.1242072


https://github.com/thomasp85/densityClust



Density Peaks 

https://github.com/lanbing510/DensityPeakCluster?msclkid=1fd85fbab28f11ec870cbd2dc3baebd7
